# Model Training and Evaluation

This notebook prepares the environment by installing necessary libraries, importing modules, mounting Google Drive, and setting up directories. It then loads and cleans the dataset, applying a text cleaning function and setting up label mappings. To address class imbalance, the training data is oversampled. The datasets are then tokenized using the DistilBERT tokenizer. A function for computing evaluation metrics such as accuracy, precision, recall, and F1-score is defined. The DistilBERT model for sequence classification is loaded, and training parameters are defined. The model is then initialized and trained using the Hugging Face Trainer on the prepared datasets. After training, the fine-tuned model and tokenizer are saved to Google Drive. Finally, the trained model is evaluated on the test set, a confusion matrix is generated, and training and validation metrics are visualized.

In [ ]:
# =========================
# Step 1 – Install necessary packages (run once)
# =========================
!pip install transformers datasets scikit-learn wandb imblearn


# =========================
# Step 2 – Imports
# =========================
from transformers import AutoTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import wandb
import os


# =========================
# Step 3 – Log in to Weights & Biases (run once)
# =========================
wandb.login()


# =========================
# Step 4 – Mount Google Drive (if using Colab)
# =========================
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/datasets/'
os.makedirs(drive_path, exist_ok=True)


# =========================
# Step 5 – Load and clean dataset
# =========================
dataset = load_dataset("cnamuangtoun/resume-job-description-fit")

def clean_text(text):
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\b\d{10,}\b', '', text)
    text = re.sub(r'\d+\s+\w+\s+(Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr)\b', '', text, flags=re.IGNORECASE)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

label_to_id = {'No Fit': 0, 'Potential Fit': 1, 'Good Fit': 2}
id2label = {v: k for k, v in label_to_id.items()}

def preprocess_batch(batch):
    batch['resume_text'] = [clean_text(t) for t in batch['resume_text']]
    batch['job_description_text'] = [clean_text(t) for t in batch['job_description_text']]
    batch['label_id'] = [label_to_id[l] for l in batch['label']]
    return batch

dataset = dataset.map(preprocess_batch, batched=True)


# =========================
# Step 6 – Oversample training data before tokenization
# =========================
df_train = pd.DataFrame(dataset['train'])
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df_train[['resume_text', 'job_description_text']], df_train['label_id'])
df_train_resampled = pd.DataFrame(X_resampled)
df_train_resampled['label_id'] = y_resampled
print("✅ Resampled class distribution:")
print(df_train_resampled['label_id'].value_counts().sort_index().rename(index=id2label))
df_train_resampled.to_csv(drive_path + "resampled_train.csv", index=False)

# Convert to Hugging Face dataset
resampled_train_dataset = Dataset.from_pandas(df_train_resampled)


# =========================
# Step 7 – Tokenization
# =========================
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["resume_text"],
        example["job_description_text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

# Tokenize datasets
tokenized_train = resampled_train_dataset.map(tokenize_function, batched=True)
tokenized_test = dataset['test'].map(tokenize_function, batched=True)

# Remove original text columns
tokenized_train = tokenized_train.remove_columns(["resume_text", "job_description_text"])
tokenized_test = tokenized_test.remove_columns(["resume_text", "job_description_text"])

# Rename columns to 'label' ensuring no conflicts
tokenized_train = tokenized_train.rename_column("label_id", "label")

# Fix for test set: remove string 'label' if exists before renaming
if "label" in tokenized_test.column_names and tokenized_test.features["label"].dtype == "string":
    tokenized_test = tokenized_test.remove_columns("label")

tokenized_test = tokenized_test.rename_column("label_id", "label")


# =========================
# Step 8 – Define metrics function
# =========================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    p_class, r_class, f_class, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1, 2])
    metrics = {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}
    for i, label in id2label.items():
        key = label.replace(" ", "_").lower()
        metrics[f'precision_{key}'] = p_class[i]
        metrics[f'recall_{key}'] = r_class[i]
        metrics[f'f1_{key}'] = f_class[i]

    step = trainer.state.global_step if hasattr(trainer.state, "global_step") else 0
    print(f"\n🔎 Step {step}")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")
    return metrics


# =========================
# Step 9 – Load model
# =========================
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


# =========================
# Step 10 – Training arguments
# =========================
training_args = TrainingArguments(
    output_dir="./results_distilbert",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    do_eval=True,
    seed=42,
    disable_tqdm=False,
    no_cuda=not torch.cuda.is_available(),
    report_to="wandb",
    run_name="resume-fit-distilbert-final"
)


# =========================
# Step 11 – Initialize Trainer
# =========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)


# =========================
# Step 12 – Train the model
# =========================
trainer.train()


# =========================
# Step 13 – Save model and tokenizer
# =========================
save_path = "/content/drive/MyDrive/datasets/fine_tuned_distilbert_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("✅ Model and tokenizer saved to Google Drive!")
print(f"Saved to: {save_path}")


# =========================
# Step 14 – Evaluation predictions
# =========================
eval_preds = trainer.predict(tokenized_test)


# =========================
# Step 15 – Confusion matrix
# =========================
cm = confusion_matrix(eval_preds.label_ids, eval_preds.predictions.argmax(-1), labels=[0, 1, 2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Fit", "Potential Fit", "Good Fit"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()


# =========================
# Step 16 – Plot training & validation metrics
# =========================
history = trainer.state.log_history
train_loss = [entry['loss'] for entry in history if 'loss' in entry]
eval_logs = [entry for entry in history if 'eval_loss' in entry]
eval_loss = [entry['eval_loss'] for entry in eval_logs]
eval_accuracy = [entry['eval_accuracy'] for entry in eval_logs]
eval_f1 = [entry['eval_f1'] for entry in eval_logs]
steps_train = [entry['step'] for entry in history if 'loss' in entry]
steps_eval = [entry['step'] for entry in eval_logs]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(steps_train, train_loss, label='Training Loss')
plt.plot(steps_eval, eval_loss, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(steps_eval, eval_accuracy, label='Validation Accuracy')
plt.plot(steps_eval, eval_f1, label='Validation F1')
plt.xlabel('Steps')
plt.ylabel('Score')
plt.title('Validation Accuracy and F1')
plt.legend()
plt.tight_layout()
plt.show()


# =========================
# Step 17 – Plot per-class F1
# =========================
plt.figure(figsize=(10, 6))
for i, label in id2label.items():
    key = label.replace(" ", "_").lower()
    f1_scores = [entry.get(f'eval_f1_{key}', None) for entry in eval_logs]
    plt.plot(steps_eval, f1_scores, label=f'F1: {label}')

plt.xlabel('Steps')
plt.ylabel('F1 Score')
plt.title('Per-Class Validation F1 Over Time')
plt.legend()
plt.show()

# Model Inference (Prediction)
This cell loads the trained model and tokenizer from Google Drive and defines a function to predict the fit score between a resume and a job description.

In [ ]:
# =========================
# Step 1 – Setup and Imports
# =========================
# Install if needed (skip if already installed)
!pip install transformers

# Imports
from transformers import AutoTokenizer, DistilBertForSequenceClassification
import torch
import torch.nn.functional as F
import os


# =========================
# Step 2 – Mount Google Drive
# =========================
from google.colab import drive
drive.mount('/content/drive')


# =========================
# Step 3 – Load model and tokenizer
# =========================
model_path = "/content/drive/MyDrive/datasets/fine_tuned_distilbert_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# =========================
# Step 4 – Label mapping
# =========================
id2label = {0: "No Fit", 1: "Potential Fit", 2: "Good Fit"}


# =========================
# Step 5 – Inference function
# =========================
def predict_fit(resume_text, job_desc_text):
    inputs = tokenizer(
        resume_text,
        job_desc_text,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        pred_label = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_label].item()
        good_fit_prob = probs[0][2].item()  # Probability of 'Good Fit'

    print("📄 Resume:")
    print(resume_text.strip()[:200] + "...\n")
    print("📝 Job Description:")
    print(job_desc_text.strip()[:200] + "...\n")
    print(f"🔍 Predicted Fit: **{id2label[pred_label]}** with {confidence:.2%} confidence.")

    print("\n📊 Probability Scores:")
    for i, prob in enumerate(probs[0]):
        print(f"  - {id2label[i]}: {prob.item():.2%}")

    print(f"\n⭐ Match Score (Good Fit Probability): {good_fit_prob:.2%}")

    print("="*80)

    return {
        "pred_label": pred_label,
        "confidence": confidence,
        "good_fit_prob": good_fit_prob
    }


# =========================
# Step 6 – Test Samples
# =========================
resume_no_fit = """
Experienced barista with 4 years in food service. Skilled in preparing espresso-based drinks, operating POS systems, and delivering high customer satisfaction in fast-paced cafés. Seeking to advance in hospitality industry.
"""

job_no_fit = """
We are hiring a Senior Machine Learning Engineer to join our FinTech team. Required skills include Python, TensorFlow, cloud deployment (AWS or Azure), and at least 3 years of experience in ML model training and MLOps pipelines.
"""

resume_potential = """
Recent graduate with a degree in Data Science. Completed coursework in machine learning, statistics, and Python. Completed a capstone project building a recommendation system. Eager to apply skills in an internship setting.
"""

job_potential = """
Looking for a junior data analyst to support data processing and visualization efforts. Experience with SQL, data dashboards, and machine learning concepts preferred.
"""

resume_good = """
Data analyst with 3 years of experience analyzing customer behavior using Python, SQL, and Tableau. Delivered weekly insights to marketing team, improving campaign performance by 18%. Experienced with scikit-learn, pandas, and cloud tools.
"""

job_good = """
We are seeking an experienced data analyst to join our customer insights team. Must have strong skills in Python, SQL, and data visualization (Tableau or Power BI). Knowledge of machine learning libraries is a plus.
"""

# Put your samples in a list of tuples
samples = [
    (resume_no_fit, job_no_fit),
    (resume_potential, job_potential),
    (resume_good, job_good)
]

match_scores = []

# Run predictions for each sample, print details, and store match scores
for i, (res, job) in enumerate(samples, 1):
    print(f"\n=== Sample {i} ===")
    result = predict_fit(res, job)
    match_scores.append(result['good_fit_prob'])

# Summary of match scores
print("\nSummary of Match Scores per sample:")
for i, score in enumerate(match_scores, 1):
    print(f"Sample {i} Good Fit Match Score: {score:.2%}")

Mounted at /content/drive

=== Sample 1 ===
📄 Resume:
Experienced barista with 4 years in food service. Skilled in preparing espresso-based drinks, operating POS systems, and delivering high customer satisfaction in fast-paced cafés. Seeking to advance i...

📝 Job Description:
We are hiring a Senior Machine Learning Engineer to join our FinTech team. Required skills include Python, TensorFlow, cloud deployment (AWS or Azure), and at least 3 years of experience in ML model t...

🔍 Predicted Fit: **Potential Fit** with 62.98% confidence.

📊 Probability Scores:
  - No Fit: 36.39%
  - Potential Fit: 62.98%
  - Good Fit: 0.62%

⭐ Match Score (Good Fit Probability): 0.62%

=== Sample 2 ===
📄 Resume:
Recent graduate with a degree in Data Science. Completed coursework in machine learning, statistics, and Python. Completed a capstone project building a recommendation system. Eager to apply skills in...

📝 Job Description:
Looking for a junior data analyst to support data processing and visualiz